In [ ]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#### variabili globali
#### GIUSEPPE
image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'

In [ ]:
dataset = pd.read_csv(csv_completo, encoding = "ISO-8859-1")
batch = 1000

def my_train_batch(dataset=dataset, batch=500, classifier='classifier1.h5'):
    size = len(dataset)
    epoch=0
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_test1, y_train, y_test1 = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.40)
        del X_test1, y_test1
        
        train_img = np.asarray(load_images(X_train))
        test_img = np.asarray(load_images(X_test))  # NB non sono mai usate, pensare se servano oppure no
        val_img = np.asarray(load_images(X_val))
        
        
        #X_bad_id = X_bad_id.reshape((X_bad_id.shape[0], 1))
        cl1 = k.models.load_model('classifier2.h5')
        cl1.fit(train_img, y_train, epochs=1, shuffle=True, validation_data=(val_img, y_val), verbose=2)
        cl1.save('classifier1.h5')
        del X_train, y_train, X_test, y_test, X_val, y_val, train_img, test_img, val_img, classifier
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples
        epoch = epoch + 1
        print("fine epoca " + epoch + ";\trestano " + size + "campioni nel dataset")
    return

# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join("..\skin-cancer-mnist-ham10000\HAM10000_images_part_1", img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return images

def new_classifier1():
    input_img = Input(shape=(450, 600, 3))  # 3x600x450 image RGB 
    #print (tf.shape(input_img))
    x = Conv2D(20, (5, 5), activation='relu', padding='same')(input_img) # 20x450x600
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 20x225x300
    #print (tf.shape(x))
    x = Conv2D(140, (3, 3), activation='relu', padding='same')(x) # 140x75x100
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 40x38x50
    #print (tf.shape(x))
    x = Conv2D(50, (3, 3), activation='relu', padding='same')(x) # 50x38x50
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 50x19x25
    #print (tf.shape(x))
    x = Flatten()(x) # qui la x diventa monodimensionale
    #print (tf.shape(x))
    # da qualche parte ci va il numero delle label?
    encoded = Dense(7, activation='softmax')(x)
    classifier = k.models.Model(input_img, encoded)   #questo è il nostro base_estimator, compile configura per il training
    classifier.compile(optimizer='adadelta', loss='binary_crossentropy')
    #classifier.compile(optimizer='adadelta', loss='categorical_crossentropy') # <- variante che non risolve il problema del dense
    return classifier

In [ ]:
### inizializzazione classificatore sul disco
cl1 = new_classifier1()
cl1.save('classifier2.h5')

In [ ]:
samples = dataset.sample(n=2000)
my_train_batch(dataset=samples, batch=batch)